# **API's**

APIs stand for Application Programming Interface. The advent of these interfaces into Government or company databases has made considerable advancements in data accessibility for the common analyst. As an individual interested in the food sector I have managed to get enormous access in food movements through the use of API's. In fact they have made data harvesting considerably easier. They can also be used as programming interfaces. So for example you can use the Google rest API to interface youe code with google drive. This allows you to expand the functionality of your programs greatly. The best way to think of them is they give your little PC the power of the cloud to complete some action. The great thing is you don't have use the same programming language as the host machine. In fact every time you use your browser you are actually using an API. The following example pulls data from the USDA( US deptartment of agriculture). You should go to this [link](https://fdc.nal.usda.gov/api-key-signup.html) to obtain a key.


In [1]:
!pip install pycurl



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 20.4 MB/s eta 0:00:00


In [2]:
#curl -H "Content-Type:application/json"
#-d '{"generalSearchInput":"Cheddar cheese"}'
#URL = "http://maps.googleapis.com/maps/api/geocode/json"

import requests

#target_url = "https://api.nal.usda.gov/fdc/v1/search"
#data = {'generalSearchInput':'almond'}
#params = {"x-api-key":'58QPvSPOC8yl1D4QXuAzlsathEqxROkjxbWlmLbN'}
#response = requests.post(target_url, headers=headers, data=params,json=data).json()

params = {'api_key': '58QPvSPOC8yl1D4QXuAzlsathEqxROkjxbWlmLbN'}
data = {'generalSearchInput': 'chia'}
response = requests.post(
    r'https://api.nal.usda.gov/fdc/v1/search',
    params=params,
    json=data
)
print(response.json())

print(response)

{'totalHits': 3549, 'currentPage': 1, 'totalPages': 71, 'pageList': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'foodSearchCriteria': {'generalSearchInput': 'chia', 'pageNumber': 1, 'numberOfResultsPerPage': 50, 'pageSize': 50, 'requireAllWords': False}, 'foods': [{'fdcId': 2106509, 'description': 'CHIA', 'dataType': 'Branded', 'gtinUpc': '897034002342', 'publishedDate': '2021-10-28', 'brandOwner': 'Global Juices and Fruits, LLC', 'brandName': "NATURE'S EARTHLY CHOICE", 'ingredients': '100% PURE NATURAL SALVIA HISPANICA L.', 'marketCountry': 'United States', 'foodCategory': 'Other Grains & Seeds', 'modifiedDate': '2020-11-19', 'dataSource': 'LI', 'packageWeight': '12 oz/340 g', 'servingSizeUnit': 'g', 'servingSize': 32.0, 'householdServingFullText': '3 Tbsp', 'tradeChannels': ['NO_TRADE_CHANNEL'], 'allHighlightFields': '', 'score': 1217.4064, 'microbes': [], 'foodNutrients': [{'nutrientId': 1003, 'nutrientName': 'Protein', 'nutrientNumber': '203', 'unitName': 'G', 'derivationCode': 'LCCS', 'deriv

You will probably note the POST request in the request object. This seems strange but the site expects you to post the data in order to get a response.

This following content provides recipes for loading and saving data from external sources to google drive using APIs. Have a look and pick the ones that are most useful for you.

# Google Drive

You can access files in Drive in a number of ways, including:
- Mounting your Google Drive in the runtime's virtual machine
- Using a wrapper around the API such as [PyDrive](https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html)
- Using the [native REST API](https://developers.google.com/drive/v3/web/about-sdk)



Examples of each are below.

## Mounting Google Drive locally

The example below shows how to mount your Google Drive on your runtime using an authorization code, and how to write and read files there. Once executed, you will be able to see the new file (`foo.txt`) at [https://drive.google.com/](https://drive.google.com/).

Note this only supports reading, writing, and moving files; to programmatically modify sharing settings or other metadata, use one of the other options below.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive

The examples below demonstrate authentication and file upload/download using PyDrive. More examples are available in the [PyDrive documentation](https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html).

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authenticate and create the PyDrive client.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Create and upload a text file.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1j6OdLlhMgjXfHYkyeHIM_egySSOmEsnk


Load a file by ID and print its contents.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive REST API

In order to use the Drive API, we must first authenticate and construct an API client.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

With this client, we can use any of the functions in the [Google Drive API reference](https://developers.google.com/drive/v3/reference/). Examples follow.


### Creating a new Drive file with data from Python

First, create a local file to upload.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Upload it using the [`files.create`](https://developers.google.com/drive/v3/reference/files/create) method. Further details on uploading files are available in the [developer documentation](https://developers.google.com/drive/v3/web/manage-uploads).

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1O68nCgHOzoIbwtvsZnj38wXDbvzCMO4z


After executing the cell above, you will see a new file named 'Sample file' at [https://drive.google.com/](https://drive.google.com/).

### Downloading data from a Drive file into Python

Download the file we uploaded above.

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


In order to download a different file, set `file_id` above to the ID of that file, which will look like "1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz".